Install Required Libraries
We install:
- **medmnist** → For PneumoniaMNIST dataset
- **timm** → For pretrained DenseNet121 model
- **torch / torchvision** → Deep learning framework
- **tqdm** → Progress bars during training

In [3]:
!pip install medmnist timm -q

# 2. Import Required Libraries

We import:
- PyTorch modules for model building and training
- torchvision transforms for preprocessing
- timm for pretrained DenseNet121
- PneumoniaMNIST dataset from MedMNIST
- DataLoader for batching

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import timm
import numpy as np
from medmnist import PneumoniaMNIST
from torch.utils.data import DataLoader
from tqdm import tqdm

# 3. Device Configuration

Automatically selects:
- GPU if available
- Otherwise CPU

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# 4. Data Preprocessing

Steps:
- Resize images to 224×224 (required by DenseNet)
- Convert grayscale to 3-channel (ImageNet models expect 3 channels)
- Convert to Tensor
# 5. Load PneumoniaMNIST Dataset

Dataset splits:
- Train
- Test

Each sample:
- Image
- Label (0 = Normal, 1 = Pneumonia)

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

train_dataset = PneumoniaMNIST(split='train', transform=transform, download=True)
test_dataset = PneumoniaMNIST(split='test', transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# 6. Load Pretrained DenseNet121

We:
- Load pretrained DenseNet121 (ImageNet)
- Replace final classifier layer
- Output size = 2 classes (Normal, Pneumonia)

In [9]:
model = timm.create_model('densenet121', pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, 2)
model = model.to(device)

model.safetensors:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

# 7. Define Loss Function and Optimizer

Loss:
- CrossEntropyLoss (for multi-class classification)

Optimizer:
- Adam (learning rate = 1e-4)
# 8. Model Training

Training loop:
- Forward pass
- Compute loss
- Backpropagation
- Update weights

Currently trained for 2 epochs (can increase for better accuracy).

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

model.train()
for epoch in range(2):   # increase for better performance
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.squeeze().long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

100%|██████████| 148/148 [00:51<00:00,  2.85it/s]


Epoch 1, Loss: 0.2010


100%|██████████| 148/148 [00:51<00:00,  2.86it/s]

Epoch 2, Loss: 0.0827


# 9. Prediction Function

This function:
- Takes one image
- Runs inference
- Returns predicted class and confidence score

In [11]:
def predict_image(image_tensor):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.unsqueeze(0).to(device)
        output = model(image_tensor)
        probs = torch.softmax(output, dim=1)
        confidence, pred = torch.max(probs, 1)
    return pred.item(), confidence.item()

# 10. Radiology Report Generator

Generates a structured radiology-style report based on:

- Predicted class
- Model confidence

⚠️ Note: This is AI-generated and not a medical diagnosis.

In [12]:
def generate_radiology_report(pred, confidence):

    if pred == 1:
        report = f"""
CHEST X-RAY REPORT

Findings:
- Increased radiographic opacity observed in lung fields.
- Patchy consolidation pattern noted.
- No obvious pleural effusion.
- Cardiomediastinal silhouette appears within normal limits.

Impression:
Findings are consistent with Pneumonia.

Model Confidence:
{confidence*100:.2f}%

Note:
This report is AI-generated based on image classification and should not replace clinical evaluation.
"""
    else:
        report = f"""
CHEST X-RAY REPORT
Findings:
- Lung fields appear clear.
- No focal consolidation detected.
- No visible pleural effusion.
- Cardiomediastinal silhouette within normal limits.

Impression:
No radiographic evidence of Pneumonia.

Model Confidence:
{confidence*100:.2f}%

Note:
This report is AI-generated based on image classification and should not replace clinical evaluation.
"""

    return report

# 11. Testing on Sample Image

In [16]:
image, label = test_dataset[0]
pred, conf = predict_image(image)

report = generate_radiology_report(pred, conf)
print(report)


CHEST X-RAY REPORT

Findings:
- Increased radiographic opacity observed in lung fields.
- Patchy consolidation pattern noted.
- No obvious pleural effusion.
- Cardiomediastinal silhouette appears within normal limits.

Impression:
Findings are consistent with Pneumonia.

Model Confidence:
99.96%

Note:
This report is AI-generated based on image classification and should not replace clinical evaluation.



Generate 10 Sample Reports

In [17]:
def generate_advanced_radiology_report(pred, confidence):

    # Determine certainty wording
    if confidence > 0.85:
        certainty = "Findings are highly suggestive of"
        recommendation = "Clinical correlation is recommended."
    elif confidence > 0.6:
        certainty = "Findings are suggestive of"
        recommendation = "Recommend clinical evaluation and possible follow-up imaging."
    else:
        certainty = "Findings are inconclusive but may indicate"
        recommendation = "Further radiological assessment is strongly advised."

    if pred == 1:
        diagnosis = "Pneumonia"
        findings = """
- Increased radiographic opacity in lung zones.
- Patchy airspace consolidation.
- No significant pleural effusion.
"""
    else:
        diagnosis = "No Acute Cardiopulmonary Abnormality"
        findings = """
- Clear lung fields.
- No focal consolidation.
- Cardiomediastinal silhouette within normal limits.
"""

    report = f"""
CHEST X-RAY REPORT
----------------------------------------

Technique:
- Single frontal chest radiograph.

Findings:
{findings}

Impression:
{certainty} {diagnosis}.

Model Confidence:
{confidence*100:.2f}%

Recommendation:
{recommendation}

Disclaimer:
AI-generated report for research purposes only.
"""

    return report

In [18]:
import random

model.eval()

indices = random.sample(range(len(test_dataset)), 10)

for i, idx in enumerate(indices):
    image, true_label = test_dataset[idx]

    pred, conf = predict_image(image)
    report = generate_advanced_radiology_report(pred, conf)

    print("="*80)
    print(f"Sample {i+1}")
    print(f"True Label: {true_label.item()} | Predicted: {pred}")
    print(report)

Sample 1
True Label: 0 | Predicted: 1

CHEST X-RAY REPORT
----------------------------------------

Technique:
- Single frontal chest radiograph.

Findings:

- Increased radiographic opacity in lung zones.
- Patchy airspace consolidation.
- No significant pleural effusion.


Impression:
Findings are highly suggestive of Pneumonia.

Model Confidence:
94.77%

Recommendation:
Clinical correlation is recommended.

Disclaimer:
AI-generated report for research purposes only.

Sample 2
True Label: 1 | Predicted: 1

CHEST X-RAY REPORT
----------------------------------------

Technique:
- Single frontal chest radiograph.

Findings:

- Increased radiographic opacity in lung zones.
- Patchy airspace consolidation.
- No significant pleural effusion.


Impression:
Findings are highly suggestive of Pneumonia.

Model Confidence:
99.99%

Recommendation:
Clinical correlation is recommended.

Disclaimer:
AI-generated report for research purposes only.

Sample 3
True Label: 1 | Predicted: 1

CHEST X-RAY